In [1]:
# L'obiettivo è passare da testo libero a una rappresentazione numerica che consente la ricerca semantica, usando transformer e vector database
# Dalla descrizione testuale ai vettori: si vogliono trovare libri simili a una certa idea, che esprimo tramite query
# Per fare ciò, serve un modo per tradurre il significato del testo in una forma matematica, così da confrontare frasi che utilizzano parole diverse
# Questo processo si chiama embedding, alla base della semantic search (confronta query con descrizione di libri)

# Embedding: rappresentazione numerica (vettore) che cattura il significato di una parola, frase o documento
# Il vettore consente di calcolare quanto due testi sono simili, in modo matematico
# Avendo a disposizione un dataset di libri molto grande, non costruisco gli embedding manualmente, ma uso modelli di embedding preaddestrati
# Questi modelli imparano dal contesto d'uso delle parole

# Distinzione:
# - Word embedding: mappano singole parole in vettori
# - Document embedding: rappresentano intere frasi (es. descrizione di un libro) in un singolo vettore

# Modelli attuali si basano sui transformer, generando document embeddings che tengono conto del contesto completo
# Usano il meccanismo di self-attention per dare peso alle parole più rilevanti nella comprensione del testo (tenendo conto anche del loro posizionamento nella frase)
# L’architettura è composta da:
# - Encoder: creano l'embedding di un testo (es. la query fornita)
# - Decoder: produce testo

# Ottenuti i document embeddings delle descrizioni dei libri, quesi sono salvati in un vector store (es. Chroma)

# In questo compito di semantic search, il modello scelto utilizza solo l'encoder, che trasforma sia:
# - La query dell’utente 
# - Ogni descrizione di libro nel database in embedding vettoriali
# Il modello poi confronta questi vettori (con cosine similarity) per trovare quelli più vicini
# (Potrebbe servire il decoder per un modello che genera una descrizione, non semplicemente la cerca come in questo caso)

# Es. trovare libri simili alla query "A book about the Roman Empire"
# - Il testo viene trasformato in un embedding
# - Il modello di embedding calcola la similarità tra questo embedding e tutti quelli nel vector store (con cosine similarity)
# - Si ottengono i libri più vicini semanticamente (anche senza contenere esattamente "Roman Empire")
# - Si recuperano le informazioni interessanti per l'utente tramite identificativo

In [9]:
# Langchain: framework che permette molte applicazioni per gli LLM, utilizzato per fare la vector search
# Pipeline: Document Loader -> Text Splitter -> Storage -> Retrieval -> LLM

from langchain_community.document_loaders import TextLoader
# Prende il testo delle descrizioni e le carica, convertendole in un formato utilizzabile da Langchain

from langchain_text_splitters import CharacterTextSplitter
# Divide le descrizioni in chunck significativi, uno per ogni libro 

#from langchain_openai import OpenAIEmbeddings 
from langchain.embeddings import HuggingFaceBgeEmbeddings
# Per convertire i chunk in document embedding
# Il modello OpenAIEmbedding è a pagamento dopo il periodo di prova, quindi utilizzo un modello di HuggingFace

from langchain_chroma import Chroma
# Per racchiudere tutto in un database vettoriale 

In [11]:
# Setting di un ambiente adatto all'utilizzo di queste applicazioni AI
# Il modello di embedding è utilizzabile tramite chiamate API: è necessario definire delle API key per connetterle all'account di openAI
# Per farlo utilizzo il pacchetto dot-env, che crea un file .env in cui inserire le chiavi 

'''
from dotenv import load_dotenv
load_dotenv()

# Controlla se la chiave API è presente
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found")
'''

# Se si usano modelli HuggingFace tramite HuggingFaceBgeEmbeddings, il modello viene scaricato e usato localmente
# Non serve quindi fare chiamate API esterne 

In [13]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [14]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883: A NOVEL THAT READERS and critic...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897: Lewis' work on the nature of lo...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5084,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...
5085,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014: This book tells the tale of a m...
5086,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...
5087,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...


In [15]:
# Per creare il vector dataset è fondamentale la feature "tagged_description"
books["tagged_description"]

# In risposta alle query poste al vector database, sono fornite le descrizioni di libri
# Le informazioni più utili da fornire agli utenti sono in realtà titoli e autori
# Quello che si può fare è, quindi, filtrare books dai risultati dalla compatibilità della query con il vector database
# Per facilitare questa operazione, la variabile "tagged_description" permette che dalle descrizioni dei libri si possa accedere ad altre informazioni tramite "isbn13"

0       9780002005883: A NOVEL THAT READERS and critic...
1       9780002261982: A new 'Christie for Christmas' ...
2       9780006178736: A memorable, mesmerizing heroin...
3       9780006280897: Lewis' work on the nature of lo...
4       9780006280934: "In The Problem of Pain, C.S. L...
                              ...                        
5084    9788172235222: On A Train Journey Home To Nort...
5085    9788173031014: This book tells the tale of a m...
5086    9788179921623: Wisdom to Create a Life of Pass...
5087    9788185300535: This collection of the timeless...
5088    9789027712059: Since the three volume edition ...
Name: tagged_description, Length: 5089, dtype: object

In [17]:
# Il TextLoader method di Langchain non funziona con i dataframe di Pandas
# Si salva quindi il contenuto di "tagged_description" dentro un file testo con elemento di separazione per ogni nuova linea
# Non sono memorizzati index e header, ma solo testo, e si presentano come frasi "isbn: description"

books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [18]:
# Prima cosa da fare è ottenere le descrizioni separate 
# Si fa il load del contenuto di "tagged_description" e si usa successivamente lo splitter di testo

raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n") # Caricamento dello splitter
# Parametri di CharacterTextSplitter:
# - chunk_overlap: se posto a 0, indica che non si vuole che i chunk siamo sovrapposti (perchè i titoli sono separati)
# - chunk_size: se posto a 0, indica che la suddivisione dei chunk priorizza la separazione su quello indicato da separator
# - separator: indica il simbolo che distingue elementi distinti, che si vogliono separare.
#   La separazione avviene al simbolo "\n", ovvero al termine della descrizione di un libro (e passaggio al prossimo)

documents = text_splitter.split_documents(raw_documents) # Applicazione dello splitter sui documenti caricati
# L'operazione dà warning sulla lunghezza del chunk per i motivi indicati sopra

Created a chunk of size 1169, which is longer than the specified 0
Created a chunk of size 1215, which is longer than the specified 0
Created a chunk of size 374, which is longer than the specified 0
Created a chunk of size 310, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 961, which is longer than the specified 0
Created a chunk of size 189, which is longer than the specified 0
Created a chunk of size 844, which is longer than the specified 0
Created a chunk of size 297, which is longer than the specified 0
Created a chunk of size 198, which is longer than the specified 0
Created a chunk of size 882, which is longer than the specified 0
Created a chunk of size 1089, which is longer than the specified 0
Created a chunk of size 1190, which is longer than the specified 0
Created a chunk of size 305, which is longer than the specified 0
Create

In [19]:
# Verifico che l'operazione sia andata a buon fine caricando il primo documento

documents[0] # Ritorna la prima descrizione, come atteso

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [20]:
# Inizializzazione del modello di embeddings

# embeddings = OpenAIEmbeddings(api_key="OPENAI_API_KEY")
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\mazze\AppData\Local\Temp\ipykernel_11792\1954680622.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [35]:
# Creazione dei document embedding a partire dai document e salvataggio in un vector database
db_books = Chroma.from_documents(documents, embedding=embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory=None)

In [37]:
# Esempio di query
query = "A book to teach children about the nature"
docs = db_books.similarity_search_with_score(query, k=5)
print(docs)

# - k: numero massimo di documenti che devono essere restituiti
# - similarity_search_with_query: cerca libri in base alla similarità tra query ed elementi nel database
#   I risultati forniti hanno un punteggio di similarità associato (più vicino è a 0, più la descrizione rispetta la query fornita) 

[(Document(id='2d4ae2cb-e686-4a51-8324-7bb34e68f9fe', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069: Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), 0.888763964176178), (Document(id='1f7eeb0e-76fe-42af-87c1-c4874a6a1cb3', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069: Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), 0.888763964176178), (Document(id='995eecaa-aa35-489d-9e3a-5e222aea9ea1', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069: Children will discover the exciting wo

In [51]:
# Ritorno informazioni utili all'utente sfruttando nella "tagged_description" la feature "isbn13"

# books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())] # Errore per isbn seguito da ":"

# Per "isbn13: description"
isbn_str = docs[0][0].page_content.split()[0].strip() # Si ottiene la prima parola (per il primo esempio nel database)
isbn_str = isbn_str.replace(":", "") # Vengono rimossi eventuali ":" alla fine
isbn_int = int(isbn_str) # Ora è sicuro convertire in intero
books[books["isbn13"] == isbn_int] # Confronto con l'isbn in books

# Si ottiene il primo libro raccomandato tramite query

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3665,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069: Children will discover the exci...


In [54]:
# Per rendere il processo più sicuro, definisco una funzione che automatizza le operazioni eseguite sopra
import re

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 5, # Ritorna 5 liibri raccomandati
) -> pd.DataFrame: 
    recs_with_scores = db_books.similarity_search_with_score(query, k = 50) # Sono dati 50 elementi dal vector database 

    # lista di tuple (isbn, score)
    data = []
    
    for rec, score in recs_with_scores:
        match = re.search(r"\b\d{10,13}\b", rec.page_content)
        if match:
            try:
                isbn_int = int(match.group()) # E' reso intero l'isbn, per poterlo confrontare successivamente con l'isbn nel dataset iniziale
                data.append((isbn_int, score)) # Tiene conto del valore score
            except ValueError:
                continue
    
    # Creazione dataframe del match
    df_matches = pd.DataFrame(data, columns=["isbn13", "similarity_score"]) 

    # Merge sicuro su isbn presenti
    result_df = books.merge(df_matches, on="isbn13")
    result_df = result_df.drop_duplicates(subset="isbn13")
    result_df = result_df.sort_values(by="similarity_score", ascending=True)
    
    return result_df.head(top_k)


In [56]:
# Esempi query
query = [
    "A book to teach children about the nature",
    "A boy and a girl in love",
    "A book about ancient times", 
    "A scientific book about human life"
]

In [67]:
recommendations = retrieve_semantic_recommendations(query[0])
print(recommendations[["isbn13", "title", "similarity_score"]])

# Ritornano i primi 5 match (con punteggio di similarità più alto)

           isbn13                                title  similarity_score
17  9780786808069  Baby Einstein: Neighborhood Animals          0.888764
32  9780802431486     The 10 Commandments of Parenting          0.893400
23  9780786808380                Baby Einstein: Babies          0.955845
20  9780786808373                 Baby Einstein: Birds          0.991680
26  9780786808397                  Baby Einstein: Dogs          0.992678
